## Evaluation

In [2]:
from llama_index.core import StorageContext,load_index_from_storage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import nest_asyncio
nest_asyncio.apply()

embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-large",device='cpu')
load_index = load_index_from_storage(StorageContext.from_defaults(persist_dir='C:/Users/sukpi/QA-vector-database/localvecstore-fullchunk-nodes'),embed_model=embed_model)

c:\Users\sukpi\anaconda3\envs\monkbot001\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
print ("Pandas version",pd.__version__)

Pandas version 2.2.1


In [4]:
csvdf = pd.read_csv('C:/Users/sukpi/patipada-bot/data-qa/fullData.csv', encoding = "utf-8")

In [5]:
csvdf.shape

(1680, 3)

In [6]:
csvdf.head()

,youtube,question,answer
0,-3mKtsIjQi8.xlsx,การทำบุญถวายสังฆทานแล้วคิดว่าอยากได้มีของทำบุญ...,คือเรื่องนี้นะต้องเข้าใจว่ามันคือพุทธศานา เราเ...
1,-3mKtsIjQi8.xlsx,ถ้าเราตั้งใจถือศีลแปดแล้วตั้งใจอดอาหารหนึ่งวัน...,ไม่อยากทานเนื้อสัตว์อาหารมังสวิรัติหาทานยากขาด...
2,-3mKtsIjQi8.xlsx,เคยได้ยินในชาดกว่าคนรักเพศเดียวกันนั้นเป็นทิฏฐ...,อันนี้ไม่ได้ถือว่าเป็นทิฐิแล้วก็ไม่ได้มีโทษอะไ...
3,-3mKtsIjQi8.xlsx,มารที่เป็นเทวบุตรมารเลือกคนมาก่อกวนไหมครับหรือ...,มีอยู่สองประเภทประเภทของการกวน มันแปลว่าผู้ขวา...
4,-3mKtsIjQi8.xlsx,นั่งสมาธิจิตเกิดความเข้าใจขึ้นมาว่าการนั่งสมาธ...,เป็นความเข้าใจที่ดีและเป็นความเข้าใจที่ถูกต้อง...


In [7]:
csvdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   youtube   1680 non-null   object
 1   question  1671 non-null   object
 2   answer    1658 non-null   object
dtypes: object(3)
memory usage: 39.5+ KB


In [8]:
df_cleaned = csvdf[csvdf['question'] != 'nan']
df_cleaned = df_cleaned[df_cleaned.applymap(lambda x: isinstance(x, str)).all(axis=1)]

C:\Users\sukpi\AppData\Local\Temp\ipykernel_7720\205483080.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df_cleaned[df_cleaned.applymap(lambda x: isinstance(x, str)).all(axis=1)]


In [9]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1658 entries, 0 to 1679
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   youtube   1658 non-null   object
 1   question  1658 non-null   object
 2   answer    1658 non-null   object
dtypes: object(3)
memory usage: 51.8+ KB


In [10]:
csvdf.tail()

,youtube,question,answer
1675,_bph8SNqh6M.xlsx,ฆราวาสบรรลุอรหันต์จะตายจริงไหมครับ,ตายจริง คือมันไม่เรียกว่าการตาย พระพุทธองค์ไม่...
1676,_bph8SNqh6M.xlsx,เมื่อเราต้องเจอคนที่เราไม่ชอบและทำให้มีอารมณ์ไ...,ตัวรู้นี้คือสติ จากการฝึกทักอารมณ์บ่อย ๆ ก็จะม...
1677,_bph8SNqh6M.xlsx,ขอความเมตตา พอจ.ต้น อธิบาย อายตนะบัพพะ ในธรรมา...,หลักของอายตนะบัพพะนี่มันอยู่ในสติปัฏฐานข้อที่ ...
1678,_bph8SNqh6M.xlsx,ส่วนมากการที่จะมาดูอายตนะได้ มันก็ต้องผ่านการด...,จริง ๆ พระพุทธองค์ตรัสตามอัธยาศัยนะ เพราะฉะนั้...
1679,_bph8SNqh6M.xlsx,ตอนนี้หนู ท่องธาตุ อยู่แต่วันหนึ่ง หนูก็ฝันไปว...,ก็ท่องธาตุในระหว่างวันไป แล้วก็ก่อนนอนก็สวดบทอ...


In [11]:
text = df_cleaned.iloc[1500:, 1].tolist()

In [12]:
print(len(text))
print(text[-1])

158
ตอนนี้หนู ท่องธาตุ อยู่แต่วันหนึ่ง หนูก็ฝันไปว่าหนูต้องท่องบท อาฏานา ฏิยปริตร 3 รอบ พระอาจารย์มีคำแนะนำว่าอย่างไรคะ


In [13]:
retrieval_engine = load_index.as_retriever(similarity_top_k = 5)
answer_list = []

for q in text:
    print(q)
    response = retrieval_engine.retrieve(q)
    answer_list.append(response)

จิตจะเข้าไปพิจารณาเทียบเคียงคําอธิบายธรรมกับทางวิทยาศาสตร์ที่จิตเคยเรียนรู้อยู่เช่นที่พระอาจารย์เคยอธิบายกับลูกศิษย์ที่ถามบางท่านจริงจริงค่ะทําให้สิ่งที่ปรากฏแก่จิตชัดเจนกว้างขวางขึ้นมากครับกราบสาธุค่ะ
ลูกอยากทราบว่าการพิจารณาตอนออกกําลังทําอย่างไรคะ
ทําไมในขณะที่ผมตั้งใจดูจิตผ่านความคิดไปตามอุบายที่ผมฟังจากพระอาจารย์ต้นพอจิตถูกรู้ว่ากําลังถูกดูอยู่สังขารความคิดมันไม่ปรากฏออกมาเลยครับแต่ก็ตั้งใจดูมันอยู่อย่างนั้นสุดท้ายก็เห็นได้เห็นว่ามันความคิดที่มันคิดที่จะไม่ให้มีความคิดอยู่แต่ใช้เวลานานมากครับที่อยู่กับความว่างไม่เห็นอะไรอาจารย์ต้นมีคําแนะนําในการดูจิตผ่านความคิดนี้ในทางปฏิบัติเพิ่มเติมไหมครับ
อยากให้พระอาจารย์อธิบายเรื่องสังฆตธรรมการปรุงแต่งที่เป็นเหตุของการเห็นว่ามีอยู่ของสรรพสิ่งในโลกสังฆธรรมดังนี้เกิดจากการปรุงแต่งของสังขารขันธ์ใช่หรือไหมคะ
สุนัขที่บ้านมักจะมานอนใกล้ใกล้เวลาสวดบทพระบริสุทธิ์ทุกครั้งเลยค่ะถ้าเราแผ่เมตตาและอุทิศบุญให้สุนัขเขาจะรับได้ไหมค่ะ
ในการพิจารณาร่างกายหกขั้นตอนสามารถพิจารณาเร็วได้หรือไม่ครับหมดลมหายใจน้ำดันดินดินปริแตกน้ำละลายดินกระดูกขาวโพลนกระดูกสลายแต่ใ

In [15]:
print(len(answer_list))

158


In [24]:
print(answer_list[5][0].text)

กราบเรียนถามพระอาจารย์ครับ ในการพิจารณาร่างกายหกขั้นตอน บางครั้งผมพิจารณาตามขั้นตอนจนเหมือนเราจมไปกับความคิดที่พิจารณนั้นเหมือนตัดการรับรู้ภายนอกไปอย่างสิ้นเชิง ไม่แน่ใจว่าอันนี้เป็นอวิชชาด้วยไหมครับเหมือนเรากับความคิดเป็นเดียวกัน การพิจารณาแบบนี้สมควรหรือไม่ครับ รบกวนขอคำแนะนำจากพระอาจารย์ด้วยครับ


In [26]:
evaluate_list = []

In [ ]:
for num_q in range(len(text)):
    ealuate_list_in = []
    print("คำถาม",num_q,":",text[num_q])
    for rank_a in range(5):
        print("คำตอบ",rank_a,":",answer_list[num_q][rank_a].text,"\n")
    relevant = input("กรอกข้อที่เกี่ยวกับคำถาม : ")
    evaluate_list_in = relevant.split()
    evaluate_list.append(evaluate_list_in)